In [1]:
# The user credential variables to access Twitter API
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

In [33]:
# if these are not included in the docker image yet
# ! pip install tweepy
# ! pip install awesome-slugify 
# slugify incompatible with python 3 - unicode

In [17]:
import tweepy
from pymongo import MongoClient
from slugify import slugify # used to slugify names - for saving files with unique names

In [18]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [19]:
api = tweepy.API(auth)

In [20]:
def get_mongo_database(db_name, host='this_mongo'):
    conn = MongoClient('this_mongo')
    return conn[db_name]

Opening up a stream and saving it to Mongo

In [21]:
from tweepy.streaming import StreamListener
import json
import time

class MyStreamListener(StreamListener):
    """ Streams tweets and saves to a MongoDB database """

    def __init__(self, api, time_limit=60):
        self.api = api
        self.start_time = time.time()
        self.limit = time_limit
        super(tweepy.StreamListener, self).__init__()
        self.col = get_mongo_database('tweets')['tweets']

    def on_data(self, tweet):
        if (time.time() - self.start_time) < self.limit:
            self.col.insert_one(json.loads(tweet))
        else:
            return False

    def on_error(self, status):
        return True # keep stream open


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
stream = tweepy.Stream(auth, MyStreamListener(api))

# Start the stream with track list of keywords
stream.filter(track=['finance', 'risks', 'singapore'])

In [22]:
tweet_db = get_mongo_database('tweets')

In [23]:
collect = tweet_db['tweets']

In [24]:
len(list(collect.find()))

219

Save to the data folder

In [36]:
import os

users = ['bono']

# Use Tweepy's built in function to get the user-time for users in the list
# Save their tweets in a file

def get_tweets(user_list, tweets=20):
    for user in user_list:
        user_timeline = api.user_timeline(screen_name=user, count=tweets)
        filename = user.lower() + '.txt'
        save_dir = '../data/' # Create directory first
        filepath = os.path.join(save_dir, filename)
        with open(filepath, 'w+') as f:
            for idx, tweet in enumerate(user_timeline):
                tweet_text = user_timeline[idx].text
                f.write(tweet_text + '\n')

In [37]:
get_tweets(users, tweets=10)